In [37]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import sys

In [3]:
def read_csv(stock):
    filepath = f"C:/Users/danie/Documents/Software/Python-Finance-QuantConnect/DATA/{stock}.csv"
    df = pd.read_csv(filepath)
    print("len(df): ", len(df))
    return df

In [68]:
def technical_indicators(df, n_ma, n_future):
    """
    calculate technical indicators of the stock
    :returns: updated dataframe
    """
        
    # parameter-dependent section
    
    # Calculate Exponential Moving Average (EMA)
    df['EMA'] = df['Log Returns'].ewm(span=n_ma, adjust=False).mean()
    # Calculate the simple moving average (SMA) for a window of n days
    df['SMA'] = df['Log Returns'].rolling(window=n_ma).mean()
    # SMA used for dependent variable
    df['y_SMA'] = df['Log Returns'].rolling(window=n_future).mean()
    df.dropna(inplace=True)
    
    
#     df['MACD_Rate_of_Change'].replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Check for NaN values
#     nan_exists = df['MACD_Rate_of_Change'].isna().any()
#     if nan_exists:
#         sys.exit("found nan")
#         analyse_nan(df)
    
    # Create series for previous and next values
#     prev_values = df['MACD_Rate_of_Change'].shift(1)
#     next_values = df['MACD_Rate_of_Change'].shift(-1)
    
    # Fill NaN values with the larger of the adjacent values
#     df['MACD_Rate_of_Change'] = np.where(
#         df['MACD_Rate_of_Change'].isna(),
#         np.maximum(prev_values, next_values),
#         df['MACD_Rate_of_Change']
#     )    
    
    # You can then fill the NaNs with a suitable value, like 0 or the mean of the column
    #     df['MACD_Rate_of_Change'].fillna(0, inplace=True)

    # If you want to fill with the mean, ensure to calculate the mean without the inf/-inf values
#     mean_value = df['MACD_Rate_of_Change'].mean()
#     df['MACD_Rate_of_Change'].fillna(mean_value, inplace=True)

    df = df.dropna(subset=['Log Returns', 'EMA'])
    df.reset_index(drop=True, inplace=True)
    return df

In [62]:
import matplotlib.pyplot as plt

def analyse_nan(df):
    # Count the number of NaN values
    nan_count = df['MACD_Rate_of_Change'].isna().sum()
    print(f"Number of NaN values: {nan_count}")
    
    nan_indices = df[df['MACD_Rate_of_Change'].isna()].index
    print("nan_indices: ", nan_indices)

    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.plot(df['MACD_Rate_of_Change'], label='MACD Rate of Change', linestyle='-', marker='o')
    plt.scatter(df[df['MACD_Rate_of_Change'].isna()].index, df[df['MACD_Rate_of_Change'].isna()]['MACD_Rate_of_Change'], color='red', label='NaN Values')
    plt.title('MACD Rate of Change with NaN Values Highlighted')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    # Histogram of NaNs
    # Creating a binary series to indicate presence of NaN
    nan_series = df['MACD_Rate_of_Change'].isna().astype(int)
    nan_series.plot(kind='hist', bins=[-0.5, 0.5, 1.5], ec='black', alpha=0.7)
    plt.title('Distribution of NaN Values in MACD Rate of Change')
    plt.xlabel('0 = Not NaN, 1 = NaN')
    plt.ylabel('Frequency')
    plt.xticks([0, 1])
    plt.show()
    
    # Stop the process
    sys.exit("Plotted areas around NaN values. Exiting the script.")

In [108]:
def pre_prepare_data(df):
    df['Daily Returns'] = df["Adj Close"].pct_change(1)
    df['Log Returns'] = np.log(1 + df['Daily Returns'])
    
    # Calculate the short-term EMA (12 periods)
    df['EMA_12'] = df['Adj Close'].ewm(span=12, adjust=False).mean()
    # Calculate the long-term EMA (26 periods)
    df['EMA_26'] = df['Adj Close'].ewm(span=26, adjust=False).mean()
    # Calculate the MACD line
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    # Calculate the Signal line
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
    # Calculate the rate of change for MACD
    df['MACD_Rate_of_Change'] = df['MACD'].pct_change()
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.loc[1:]
    
#     inf_exists = np.isinf(df['MACD_Rate_of_Change']).any()
    inf_sum = np.isinf(df['MACD_Rate_of_Change']).sum()
    if inf_sum:
        inf_indices = df[np.isinf(df['MACD_Rate_of_Change'])].index
        print("inf_indices: ", inf_indices)
        sys.exit(f"found {inf_sum} inf")
        print(f"Infinite values: {inf_exists}")

    
#     df['MACD_Rate_of_Change'].replace([np.inf, -np.inf], np.nan, inplace=True)
    # You can then fill the NaNs with a suitable value, like 0 or the mean of the column
#     df['MACD_Rate_of_Change'].fillna(0, inplace=True)
    
    # Create series for previous and next values
#     prev_values = df['MACD_Rate_of_Change'].shift(1)
#     next_values = df['MACD_Rate_of_Change'].shift(-1)
    
    # Fill NaN values with the larger of the adjacent values
#     df['MACD_Rate_of_Change'] = np.where(
#         df['MACD_Rate_of_Change'].isna(),
#         np.maximum(prev_values, next_values),
#         df['MACD_Rate_of_Change']
#     )

    # If you want to fill with the mean, ensure to calculate the mean without the inf/-inf values
#     mean_value = df['MACD_Rate_of_Change'].mean()
#     df['MACD_Rate_of_Change'].fillna(mean_value, inplace=True)

    # Encoding the crossover
    df['MACD_Crossover_Up'] = (df['MACD'] > df['Signal_Line']).astype(int)
    df['MACD_Crossover_Down'] = (df['MACD'] < df['Signal_Line']).astype(int)
    
    max_value_threshold = 1e9  # example threshold, adjust as necessary
    if df['MACD_Rate_of_Change'].abs().max() > max_value_threshold:
        print(f"Values too large found in 'MACD_Rate_of_Change' exceeding {max_value_threshold}")

    return df

In [109]:
def prepare_data(df, n_days, n_ma, n_future):
    df = technical_indicators(df, n_ma, n_future)
    
    # Calculate the indices for slicing the EMA column
    indices = np.arange((n_days - 1) * n_ma, len(df) - n_future)
    ema_indices = np.array([indices - i * n_ma for i in range(n_days)]).T

    # Extract EMAs, MACD values, and signal lines using vectorized operations
    emas = df['EMA'].values[ema_indices]
    macd_values = df['MACD'].values[indices]
    signal_lines = df['Signal_Line'].values[indices]
    macd_roc = df['MACD_Rate_of_Change'].values[indices]
    macd_crossover_up = df['MACD_Crossover_Up'].values[indices]
    macd_crossover_down = df['MACD_Crossover_Down'].values[indices]

    # Stack all features horizontally
    X = np.hstack([emas, macd_values[:, None], signal_lines[:, None], 
                   macd_roc[:, None], macd_crossover_up[:, None], macd_crossover_down[:, None]])

    # Prepare Y using vectorized slicing
    Y = df['y_SMA'].values[indices + n_future]

    return X, Y

In [110]:
def sharpe_ratio(y_true, y_pred, risk_free_rate=0):
    """
    Calculate the Sharpe Ratio for predictions.
    Here, we consider returns as the difference between true and predicted values.
    """
    # Convert lists to numpy arrays if necessary
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Calculate returns (this can be adjusted based on your specific definition)
    returns = y_true - y_pred

    # Calculate the excess returns
    excess_returns = returns - risk_free_rate

    # Calculate Sharpe Ratio
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns)

    # Handle division by zero
    sharpe_ratio = mean_excess_return / std_excess_return if std_excess_return != 0 else 0

    return sharpe_ratio


In [111]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, mean_absolute_error

def LR_MACD(df, n_days, n_ma, n_future, n_feature):  
    
    X, Y = prepare_data(df, n_days, n_ma, n_future)

    # split data
    test_size = int(len(X) * 0.2)

    # Training set
    X_train = X[:-test_size]
    y_train = Y[:-test_size]

    # Testing set
    X_test = X[-test_size:]
    y_test = Y[-test_size:]
    
    scaler = StandardScaler(with_mean=True)
    
    # Scale data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit the model on the training data
    k = max(1, n_days + n_feature)  # or any other number you want to keep
    selector = SelectKBest(f_regression, k=k).fit(X_train_scaled, y_train)
    X_train_selected = selector.transform(X_train_scaled)
    X_test_selected = selector.transform(X_test_scaled)
    
    model = LinearRegression().fit(X_train_selected, y_train)

    # n_jobs = None (number of parallelism. -1 uses all available processors)
    
    y_pred = model.predict(X_test_selected)

    # Calculate Sharpe Ratio
#     sr_score = sharpe_ratio(y_test, y_pred)
#     mse_score = mean_squared_error(y_test, y_pred)
#     mae_score = mean_absolute_error(y_test, y_pred)

    r_sq = model.score(X_test_selected, y_test)
    
    # Assuming your pipeline is named 'model' and it has been fit to your data:
    selected_features = selector.get_support()
    feature_scores = selector.scores_

    # To get the names of the selected features, if you have them:
    feature_names = np.array(_feature_names(n_days))
    selected_feature_names = feature_names[selected_features]
    selected_feature_scores = feature_scores[selected_features]
    feature_dict = dict(zip(selected_feature_names, selected_feature_scores))
#     print(f"intercept: {model.intercept_}")
#     print(f"slope: {model.coef_}")
    return r_sq, feature_dict

In [ ]:
import time

def main():
    stocks = ['AAPL','BAC','COST','C','DG','FB','HSBC','JPM']
    best = {}
    secondbest = {}

    # Start time for the whole calculation
    start_time_total = time.time()

    for stock in stocks:
        # Start time for the current stock calculation
        start_time_stock = time.time()

        df = read_csv(stock)
        df = pre_prepare_data(df)
        best[stock] = {'score': -1e9}
        secondbest[stock] = {'score': -1e9}

         # Get the ranges for the current stock
        stock_ranges = ranges[stock]
        print(f"stock_ranges: {stock_ranges}")
        days_list = list(range(*stock_ranges['days']))
        n_mas = list(range(*stock_ranges['n_ma']))
        n_futures = list(range(*stock_ranges['n_future']))
        n_features = list(range(5,-max(days_list),-1))
#         days_list = n_mas = n_futures = [1,5]
        best_n_feature = None
        secondbest_n_feature = None
        n_feature = 0
        for n_days in days_list:
            for n_ma in n_mas:
                for n_future in n_futures:
#                     print(f"days: {days}, n_ma: {n_ma}, n_future: {n_future}")
                    for n_feature in n_features:
                        score, feature_dict = LR_MACD(df.copy(),n_days, n_ma, n_future, n_feature)
                        if score > best[stock]['score']:
                            secondbest[stock] = best[stock]
                            best[stock] = {'score': score, 'days': n_days, 
                                           'n_ma': n_ma, 'n_future': n_future,
                                           'features': feature_dict}
                            best_n_feature = (max(1, n_days + n_feature), n_feature)
                        elif score > secondbest[stock]['score']:
                            secondbest[stock] = {'score': score, 'days': n_days, 
                                               'n_ma': n_ma, 'n_future': n_future,
                                               'features': feature_dict}
                            secondbest_n_feature = (max(1, n_days + n_feature), n_feature)
            print(". ", end="")
    #         print(f"{days} n_days best: {best[stock]}")
        # End time for the current stock calculation
        end_time_stock = time.time()
        print(f"\nTime for {stock}: {(end_time_stock - start_time_stock)//60:.0f} minutes"
              f" and {(end_time_stock - start_time_stock)%60:.0f} seconds.")
        print(f"best[{stock}] is: {best[stock]}")
        print(f"best n_feature is {best_n_feature}\n")
        print(f"secondbest[{stock}] is: {secondbest[stock]}")
        print(f"secondbest n_feature is {secondbest_n_feature}\n")

    end_time_total = time.time()
    print(f"Total time: {(end_time_total - start_time_total)//60:.0f} minutes"
          f" and {(end_time_total - start_time_total)%60:.0f} seconds")
    print("best:\n", best)
    print_comparison3(best, secondbest)
    
main()

len(df):  1258
stock_ranges: {'days': (14, 20, 1), 'n_ma': (12, 20, 1), 'n_future': (1, 10, 1), 'n_feature': (1, 5, 1)}


C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Up'] = (df['MACD'] > df['Signal_Line']).astype(int)
C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Down'] = (df['MACD'] < df['Signal_Line']).astype(int)


. . . . . . 
Time for AAPL: 1 minutes and 19 seconds.
best[AAPL] is: {'score': 0.08936495826001689, 'days': 15, 'n_ma': 12, 'n_future': 9, 'features': {'EMA_8': 14.68608637315489, 'MACD_crossover_up': 17.081008228845242, 'MACD_crossover_down': 17.081008228845242}}
best n_feature is (3, -12)

secondbest[AAPL] is: {'score': 0.08641182863108576, 'days': 14, 'n_ma': 12, 'n_future': 9, 'features': {'EMA_8': 14.269840584821521, 'MACD_crossover_up': 17.504749883177148, 'MACD_crossover_down': 17.504749883177148}}
secondbest n_feature is (4, -10)

len(df):  1259
stock_ranges: {'days': (4, 10, 1), 'n_ma': (8, 13, 1), 'n_future': (1, 4, 1), 'n_feature': (1, 5, 1)}


C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Up'] = (df['MACD'] > df['Signal_Line']).astype(int)
C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Down'] = (df['MACD'] < df['Signal_Line']).astype(int)


. . . . . . 
Time for BAC: 0 minutes and 8 seconds.
best[BAC] is: {'score': -0.0049832700929071105, 'days': 4, 'n_ma': 12, 'n_future': 2, 'features': {'EMA_1': 0.016025559329364707, 'EMA_2': 3.4730377253899873, 'EMA_3': 0.7846376297906182, 'EMA_4': 3.264318132973107, 'Signal_line': 0.27900788312396096, 'MACD_crossover_up': 1.893312177601801, 'MACD_crossover_down': 1.893312177601801}}
best n_feature is (7, 3)

secondbest[BAC] is: {'score': -0.00557690220020568, 'days': 5, 'n_ma': 9, 'n_future': 3, 'features': {'EMA_5': 7.177238723674945}}
secondbest n_feature is (1, -4)

len(df):  1258
stock_ranges: {'days': (13, 22, 1), 'n_ma': (15, 22, 1), 'n_future': (1, 10, 1), 'n_feature': (1, 5, 1)}


...

. . . . . . . 
Time for FB: 1 minutes and 11 seconds.
best[FB] is: {'score': 0.07134027836525303, 'days': 7, 'n_ma': 22, 'n_future': 8, 'features': {'EMA_1': 1.8796117157749375, 'EMA_2': 8.022504823082024, 'EMA_4': 1.6276426181498, 'EMA_5': 5.166189081502453, 'Signal_line': 1.3729281690204793, 'MACD_rate_of_change': 3.0869047362587962, 'MACD_crossover_up': 2.6325218711418663, 'MACD_crossover_down': 2.6325218711418663}}
best n_feature is (8, 1)

secondbest[FB] is: {'score': 0.06981580156687583, 'days': 6, 'n_ma': 17, 'n_future': 8, 'features': {'EMA_1': 2.145445651727497, 'EMA_2': 9.196088712963103, 'EMA_3': 9.508940767584555, 'EMA_5': 1.9774977588290368, 'EMA_6': 1.3116696030269521, 'Signal_line': 1.0384385915389973, 'MACD_rate_of_change': 3.2325393163815064, 'MACD_crossover_up': 4.1308296734687255, 'MACD_crossover_down': 4.1308296734687255}}
secondbest n_feature is (8, 3)

len(df):  1259
stock_ranges: {'days': (17, 30, 1), 'n_ma': (17, 26, 1), 'n_future': (15, 25, 1), 'n_feature': (1,

C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Up'] = (df['MACD'] > df['Signal_Line']).astype(int)
C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Down'] = (df['MACD'] < df['Signal_Line']).astype(int)


. . . . . . . . . . . . . 
Time for HSBC: 5 minutes and 1 seconds.
best[HSBC] is: {'score': 0.5361750890802671, 'days': 26, 'n_ma': 24, 'n_future': 19, 'features': {'EMA_7': 33.72901033008629}}
best n_feature is (1, -25)

secondbest[HSBC] is: {'score': 0.5361750890802671, 'days': 26, 'n_ma': 24, 'n_future': 19, 'features': {'EMA_7': 33.72901033008629}}
secondbest n_feature is (1, -26)

len(df):  1259
stock_ranges: {'days': (19, 25, 1), 'n_ma': (13, 20, 1), 'n_future': (13, 25, 1), 'n_feature': (1, 5, 1)}


C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Up'] = (df['MACD'] > df['Signal_Line']).astype(int)
C:\Users\danie\AppData\Local\Temp\ipykernel_10164\2114711054.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MACD_Crossover_Down'] = (df['MACD'] < df['Signal_Line']).astype(int)


. . . . . 

In [8]:
ranges = {'AAPL': {'days': (14, 20, 1), 'n_ma': (12, 20, 1), 'n_future': (1, 10, 1), 'n_feature': (1, 5, 1)},
         'BAC':   {'days': (4, 10, 1),  'n_ma': (8, 13, 1), 'n_future': (1, 4, 1), 'n_feature': (1, 5, 1)},
         'COST':  {'days': (13, 22, 1), 'n_ma': (15, 22, 1),'n_future': (1, 10, 1), 'n_feature': (1, 5, 1)},
         'C':     {'days': (1, 9, 1), 'n_ma': (1, 8, 1), 'n_future': (1, 8, 1), 'n_feature': (1, 5, 1)},
         'DG':    {'days': (10, 18, 1), 'n_ma': (10, 18, 1), 'n_future': (1, 8, 1), 'n_feature': (1, 5, 1)},
         'FB':    {'days': (1, 8, 1),  'n_ma': (12, 25, 1), 'n_future': (1, 10, 1), 'n_feature': (1, 5, 1)},
         'HSBC':  {'days': (22, 30, 1), 'n_ma': (20, 26, 1), 'n_future': (15, 25, 1), 'n_feature': (1, 5, 1)},
         'JPM':   {'days': (19, 25, 1), 'n_ma': (13, 20, 1), 'n_future': (13, 25, 1), 'n_feature': (1, 5, 1)}}

In [9]:
def sharpe_ratio(y_true, y_pred, risk_free_rate=0):
    """
    Calculate the Sharpe Ratio for predictions.
    Here, we consider returns as the difference between true and predicted values.
    """
    # Calculate returns (this can be adjusted based on your specific definition)
    returns = y_true - y_pred

    # Calculate the excess returns
    excess_returns = returns - risk_free_rate

    # Calculate Sharpe Ratio
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns)
    sharpe_ratio = mean_excess_return / std_excess_return if std_excess_return != 0 else 0

    return sharpe_ratio

In [32]:
from sklearn.feature_selection import RFECV
# from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer 

def LR_MACD_RFECV(df, n_days, n_ma, n_future, splits):  
    X, Y = prepare_data(df, n_days, n_ma, n_future)

    # Split data
    test_size = int(len(X) * 0.2)
    X_train = X[:-test_size]
    y_train = Y[:-test_size]
    X_test = X[-test_size:]
    y_test = Y[-test_size:]
    
    scaler = StandardScaler(with_mean=True)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Configure and create the RFECV model
#     estimator = LinearRegression()

    # Create a custom scorer
#     sharpe_scorer = make_scorer(sharpe_ratio, greater_is_better=True)
    
    estimator = RandomForestRegressor(n_jobs=-1)
#     estimator = LinearRegression(n_jobs=-1)
    tscv = TimeSeriesSplit(n_splits=splits) #KFold(5)

    # Use in RFECV
#     rfecv = RFECV(estimator, step=1, cv=tscv, scoring=sharpe_scorer)
    rfecv = RFECV(estimator, step=1, cv=tscv, scoring='r2')

    # Fit the model on the training data
    rfecv.fit(X_train_scaled, y_train)

    # Evaluate the model
    r_sq = rfecv.score(X_test_scaled, y_test)

    # Accessing Selected Features
    selected_features = rfecv.support_
    feature_ranking = rfecv.ranking_
    
    # Get feature names if available
    feature_names = np.array(_feature_names(n_days))
    selected_feature_names = feature_names[selected_features]
    feature_dict = dict(zip(selected_feature_names, feature_ranking[selected_features]))

    return r_sq, feature_dict

In [10]:
def _feature_names(n_days):
    # Add EMA feature names based on n_days
    ema_feature_names = [f"EMA_{day}" for day in range(1, n_days + 1)]

    # Add the names of the other features
    macd_feature_names = ['MACD_value', 'Signal_line', 'MACD_rate_of_change', 'MACD_crossover_up', 'MACD_crossover_down']
    
    return ema_feature_names + macd_feature_names

In [35]:
import time

def main():
    stocks = ['AAPL','BAC'] #,'COST','C','DG','FB','HSBC','JPM']
    best = {}
    secondbest = {}

    # Start time for the whole calculation
    start_time_total = time.time()

    for stock in stocks:
        # Start time for the current stock calculation
        start_time_stock = time.time()

        df = read_csv(stock)
        best[stock] = {'r_sq': -1e9}
        secondbest[stock] = {'r_sq': -1e9}

         # Get the ranges for the current stock
        stock_ranges = ranges[stock]
        print(f"stock_ranges: {stock_ranges}")
        days_list = list(range(*stock_ranges['days']))
        n_mas = list(range(*stock_ranges['n_ma']))
        n_futures = list(range(*stock_ranges['n_future']))
#         days_list = [11,12]
        splitss = list(range(2,5,1))
        best_splits = None
        secondbest_splits = None
        
        for days in days_list:
            for n_ma in n_mas:
                for n_future in n_futures:
                    for splits in splitss:
#                         print(f"splits: {splits}")
                        r_sq, feature_dict = LR_MACD_RFECV(df.copy(), days, n_ma, n_future, splits)
                        if r_sq > best[stock]['r_sq']:
                            secondbest[stock] = best[stock]
                            best[stock] = {'r_sq': r_sq, 'days': days, 
                                           'n_ma': n_ma, 'n_future': n_future,
                                           'features': feature_dict}
                            best_splits = splits
                        elif r_sq > secondbest[stock]['r_sq']:
                            secondbest[stock] = {'r_sq': r_sq, 'days': days, 
                                               'n_ma': n_ma, 'n_future': n_future,
                                               'features': feature_dict}
                            secondbest_splits = splits
                print(f"best[{stock}] is: {best[stock]}")
                print(f"best splits: ", best_splits)
                print(f"secondbest[{stock}] is: {secondbest[stock]}")
                print(f"secondbest splits: ", secondbest_splits)
#                 print(", ", end="")
            print(". ", end="")
    
        # End time for the current stock calculation
        end_time_stock = time.time()
        print(f"\nTime for {stock}: {(end_time_stock - start_time_stock)//60:.0f} minutes"
              f" and {(end_time_stock - start_time_stock)%60:.0f} seconds.")
        print(f"best[{stock}] is: {best[stock]}")
        print(f"best splits: ", best_splits)
        print(f"secondbest[{stock}] is: {secondbest[stock]}")
        print(f"secondbest splits: ", secondbest_splits)

    end_time_total = time.time()
    print(f"Total time: {(end_time_total - start_time_total)//60:.0f} minutes"
          f" and {(end_time_total - start_time_total)%60:.0f} seconds")
    print("best:\n", best)
    print_comparison3(best, secondbest)
    
main()

len(df):  1258
stock_ranges: {'days': (8, 15, 1), 'n_ma': (5, 15, 1), 'n_future': (3, 10, 1)}
best[AAPL] is: {'r_sq': -0.17488791650703583, 'days': 8, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1, 'MACD_crossover_down': 1}}
best splits:  4
secondbest[AAPL] is: {'r_sq': -0.2104331878774366, 'days': 8, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1, 'MACD_crossover_down': 1}}
secondbest splits:  3
best[AAPL] is: {'r_sq': -0.17488791650703583, 'days': 8, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'M

best[AAPL] is: {'r_sq': -0.13812040144327553, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1}}
best splits:  4
secondbest[AAPL] is: {'r_sq': -0.14197965945349322, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_down': 1}}
secondbest splits:  2
best[AAPL] is: {'r_sq': -0.13812040144327553, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1}}
best splits:  4
secondbest[AAPL] is: {'r_sq': -0.14197965945349322, 'days': 9, 'n_ma

best[AAPL] is: {'r_sq': -0.13812040144327553, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1}}
best splits:  4
secondbest[AAPL] is: {'r_sq': -0.14197965945349322, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_down': 1}}
secondbest splits:  2
best[AAPL] is: {'r_sq': -0.13812040144327553, 'days': 9, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1}}
best splits:  4
secondbest[AAPL] is: {'r_sq': -0.14197965945349322, 'days': 9, 'n_ma

. best[AAPL] is: {'r_sq': -0.11167323068691348, 'days': 11, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_10': 1, 'EMA_11': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1}}
best splits:  3
secondbest[AAPL] is: {'r_sq': -0.12074915947013687, 'days': 11, 'n_ma': 10, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_3': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_9': 1, 'EMA_10': 1, 'EMA_11': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1, 'MACD_crossover_up': 1, 'MACD_crossover_down': 1}}
secondbest splits:  2
best[AAPL] is: {'r_sq': -0.11167323068691348, 'days': 11, 'n_ma': 5, 'n_future': 3, 'features': {'EMA_1': 1, 'EMA_2': 1, 'EMA_4': 1, 'EMA_5': 1, 'EMA_6': 1, 'EMA_7': 1, 'EMA_8': 1, 'EMA_10': 1, 'EMA_11': 1, 'MACD_value': 1, 'Signal_line': 1, 'MACD_rate_of_change': 1}}
best splits:  3
secondbest[AAPL] is: {'r_sq': -0.12074915947013687, 'day

KeyboardInterrupt: 

In [20]:
ranges = {'AAPL': {'days': (1, 15, 1), 'n_ma': (5, 15, 1), 'n_future': (2, 8, 1)},
         'BAC':   {'days': (1, 15, 1),  'n_ma': (10, 21, 1), 'n_future': (1, 6, 1)},
         'COST':  {'days': (8, 15, 1), 'n_ma': (15, 25, 1),'n_future': (6, 14, 1)},
         'C':     {'days': (1, 10, 1), 'n_ma': (1, 11, 1), 'n_future': (1, 12, 1)},
         'DG':    {'days': (1, 21, 1), 'n_ma': (1, 21, 1), 'n_future': (1, 21, 1)},
         'FB':    {'days': (1, 9, 1),  'n_ma': (8, 21, 1), 'n_future': (4, 11, 1)},
         'HSBC':  {'days': (1, 21, 1), 'n_ma': (1, 22, 1), 'n_future': (1, 21, 1)},
         'JPM':   {'days': (1, 21, 1), 'n_ma': (1, 18, 1), 'n_future': (1, 21, 1)}}

# Ticker        R^2                 Days       n_ma       n_future
# -----------------------------------------------------------------
# AAPL      0.02480 (   0.02455)      6 (  6)     17 ( 17)         8 (        6)
# BAC       0.00026 (  -0.00018)      4 (  1)     12 ( 11)         1 (        1)
# COST      0.06092 (   0.05711)      9 (  7)     16 ( 16)        10 (       10)
# C         0.00722 (   0.00099)      2 (  1)      1 ( 20)         1 (        1)
# DG        0.01292 (   0.00944)      7 (  5)      9 ( 18)         2 (        1)
# FB        0.06486 (   0.05884)     10 (  6)     10 ( 18)         5 (        8)
# HSBC      0.01981 (   0.01821)      7 (  1)      2 (  1)         1 (        1)
# JPM       0.00436 (   0.00224)      4 (  1)     12 ( 11)         2 (        1)

In [9]:
# limited ranges for random forest
ranges = {'AAPL': {'days': (12, 15, 1), 'n_ma': (8, 10, 1), 'n_future': (4, 6, 1)},
         'BAC':   {'days': (4, 5, 1),  'n_ma': (9, 10, 1), 'n_future': (1, 3, 1)},
         'COST':  {'days': (13, 14, 1), 'n_ma': (16, 17, 1),'n_future': (9, 13, 1)},
         'C':     {'days': (2, 4, 1), 'n_ma': (1, 2, 1), 'n_future': (1, 3, 1)},
         'DG':    {'days': (13, 14, 1), 'n_ma': (1, 2, 1), 'n_future': (1, 2, 1)},
         'FB':    {'days': (2, 4, 1),  'n_ma': (17, 20, 1), 'n_future': (6, 7, 1)},
         'HSBC':  {'days': (17, 18, 1), 'n_ma': (19, 20, 1), 'n_future': (12, 13, 1)},
         'JPM':   {'days': (20, 21, 1), 'n_ma': (16, 17, 1), 'n_future': (9, 10, 1)}}

# Ticker        R^2                 Days        n_ma      n_future
# ----------------------------------------------------------------
# AAPL      0.06698 (   0.05157)     14 ( 12)      8 (  9)      5 (  4)
# BAC      -0.00593 (  -0.00593)      4 (  4)      9 (  9)      1 (  1)
# COST      0.12862 (   0.10183)     13 ( 13)     16 ( 16)     12 (  9)
# C         0.01151 (   0.00739)      3 (  2)      1 (  1)      1 (  2)
# DG        0.00447 (   0.00447)     13 ( 13)      1 (  1)      1 (  1)
# FB        0.04449 (   0.03276)      2 (  3)     17 ( 19)      6 (  6)
# HSBC      0.11487 (   0.11487)     17 ( 17)     19 ( 19)     12 ( 12)
# JPM      -0.04052 (  -0.04052)     20 ( 20)     16 ( 16)      9 (  9)

In [270]:
import cProfile
if __name__ == "__main__":
    cProfile.run('main()', 'profiling_results.out')

len(df):  1258
stock_ranges: {'days': (7, 12), 'n_ma': (2, 12), 'n_future': (1, 5)}
new best[stock]:  {'r_sq': 0.004367860912637411, 'days': 7, 'n_ma': 2, 'n_future': 1}
new best[stock]:  {'r_sq': 0.006212028567458505, 'days': 7, 'n_ma': 3, 'n_future': 3}
new best[stock]:  {'r_sq': 0.007486773938031432, 'days': 7, 'n_ma': 6, 'n_future': 1}
new best[stock]:  {'r_sq': 0.012391589435892136, 'days': 7, 'n_ma': 7, 'n_future': 1}
.new best[stock]:  {'r_sq': 0.017431723477691174, 'days': 8, 'n_ma': 3, 'n_future': 3}
..new best[stock]:  {'r_sq': 0.022319693618527814, 'days': 10, 'n_ma': 10, 'n_future': 1}
.new best[stock]:  {'r_sq': 0.023623854324563487, 'days': 11, 'n_ma': 10, 'n_future': 1}
new best[stock]:  {'r_sq': 0.025996372211829044, 'days': 11, 'n_ma': 10, 'n_future': 3}
.Time for AAPL: 2.0 minutes and 15.316807508468628 seconds.
best[AAPL] is: {'r_sq': 0.025996372211829044, 'days': 11, 'n_ma': 10, 'n_future': 3}

len(df):  1259
stock_ranges: {'days': (1, 5), 'n_ma': (1, 6), 'n_future'

In [ ]:
import pstats

# Create a Stats object
p = pstats.Stats('profiling_results.out')

# Print the statistics
# p.strip_dirs().sort_stats(-1).print_stats()
p.sort_stats('cumulative').print_stats(40)

In [269]:
ranges = {
    'AAPL': {'days': (7, 12), 'n_ma': (7, 12), 'n_future': (1, 5)},
    'BAC': {'days': (1, 11), 'n_ma': (1, 15), 'n_future': (1, 11)},
    'COST': {'days': (8, 11), 'n_ma': (21, 23), 'n_future': (21, 24)},
    'C': {'days': (1, 4), 'n_ma': (1, 12), 'n_future': (1, 4)},
    'DG': {'days': (4, 9), 'n_ma': (8, 20), 'n_future': (1, 4)},
    'FB': {'days': (1, 3), 'n_ma': (7, 10), 'n_future': (5, 10)},
    'HSBC': {'days': (1, 5), 'n_ma': (1, 3), 'n_future': (1, 3)},
    'JPM': {'days': (2, 6), 'n_ma': (11, 14), 'n_future': (1, 4)}
}
# Ticker        R^2                 Days        n_ma      n_future
# ----------------------------------------------------------------
# AAPL      0.02232 (   0.01743)     10 (  8)     10 (  3)      1 (  3)
# BAC      -0.00068 (  -0.00268)      3 (  2)      2 (  4)      1 (  1)
# COST      0.18193 (   0.17626)      9 (  9)     21 ( 21)     22 ( 21)
# C         0.00722 (  -0.00139)      2 (  1)      1 ( 10)      1 (  2)
# DG        0.00890 (   0.00539)      7 (  5)      9 ( 18)      2 (  2)
# FB        0.04299 (   0.03557)      1 (  1)      8 (  8)      8 (  6)
# HSBC      0.01823 (   0.01249)      3 (  2)      1 (  1)      1 (  1)
# JPM       0.00169 (  -0.00328)      4 (  3)     12 ( 12)      2 (  1)

In [11]:
import statistics
import math

def print_comparison3(best, secondbest):
    # Print the header
    header = f"{'Ticker':<6} {'Score':>10} {'':>13} {'Days':>6} {'':>4} {'n_ma':>6} {'':>4} {'n_future':>6}"
    print(header)
    print('-' * len(header))

    # Initialize lists to hold the values for calculating the summary statistics
    r2_values = []
    r2_values_second = []
    days_values = []
    days_values_second = []
    n_ma_values = []
    n_ma_values_second = []
    n_future_values = []
    n_future_values_second = []

    # Print each item and collect values for the summary
    for ticker in best:
        best_metrics = best[ticker]
        secondbest_metrics = secondbest.get(ticker, {})
        
        # Append values for best and second best performance
        r2_values.append(best_metrics['score'])
        r2_values_second.append(secondbest_metrics.get('score', float('nan')))
        days_values.append(best_metrics['days'])
        days_values_second.append(secondbest_metrics.get('days', float('nan')))
        n_ma_values.append(best_metrics['n_ma'])
        n_ma_values_second.append(secondbest_metrics.get('n_ma', float('nan')))
        n_future_values.append(best_metrics['n_future'])
        n_future_values_second.append(secondbest_metrics.get('n_future', float('nan')))
        
        # Format and print the line for each ticker
        line = f"{ticker:<6} {best_metrics['score']:>10.5f} ({secondbest_metrics.get('score', 'n/a'):>10.5f}) "
        line += f"{best_metrics['days']:>6} ({secondbest_metrics.get('days', 'n/a'):>3}) "
        line += f"{best_metrics['n_ma']:>6} ({secondbest_metrics.get('n_ma', 'n/a'):>3})"
        line += f" {best_metrics['n_future']:>6} ({secondbest_metrics.get('n_future', 'n/a'):>3})"
        print(line)

    # Calculate the summary statistics for best and second best performances
    def calculate_summary(values):
        # Filter out nan values for accurate calculation
        filtered_values = [v for v in values if not math.isnan(v)]
        average = statistics.mean(filtered_values)
        median = statistics.median(filtered_values)
        stdev = statistics.stdev(filtered_values) if len(filtered_values) > 1 else 0
        return average, median, stdev

    # Calculate and print the best performance summary
    r2_avg, r2_med, r2_stdev = calculate_summary(r2_values)
    days_avg, days_med, days_stdev = calculate_summary(days_values)
    n_ma_avg, n_ma_med, n_ma_stdev = calculate_summary(n_ma_values)
    n_future_avg, n_future_med, n_future_stdev = calculate_summary(n_future_values)
    
    # Calculate and print the second best performance summary
    r2_avg_second, r2_med_second, r2_stdev_second = calculate_summary(r2_values_second)
    days_avg_second, days_med_second, days_stdev_second = calculate_summary(days_values_second)
    n_ma_avg_second, n_ma_med_second, n_ma_stdev_second = calculate_summary(n_ma_values_second)
    n_future_avg_second, n_future_med_second, n_future_stdev_second = calculate_summary(n_future_values_second)


    # Print the rows for Average, Median, and StDev with the calculated values
    print("\nPerformance Summary:")
    print(f"{'Metric':<8} {'Score':>8.5} {'':>10} {'Days':>8.5} {'':>10} {'n_ma':>8.5} {'':>10} {'n_future':>8.5}")
    # Print the rows for Average, Median, and StDev with the best and second-best values
    print(f"{'Average':<8} {r2_avg:>8.5f} ({r2_avg_second:>8.5f}) {days_avg:>5.2f} ({days_avg_second:>5.2f}) {n_ma_avg:>5.2f} ({n_ma_avg_second:>5.2f}) {n_future_avg:>5.2f} ({n_future_avg_second:>5.2f})")
    print(f"{'Median':<8} {r2_med:>8.5f} ({r2_med_second:>8.5f}) {days_med:>5.2f} ({days_med_second:>5.2f}) {n_ma_med:>5.2f} ({n_ma_med_second:>5.2f}) {n_future_med:>5.2f} ({n_future_med_second:>5.2f})")
    print(f"{'StDev':<8} {r2_stdev:>8.5f} ({r2_stdev_second:>8.5f}) {days_stdev:>5.2f} ({days_stdev_second:>5.2f}) {n_ma_stdev:>5.2f} ({n_ma_stdev_second:>5.2f}) {n_future_stdev:>5.2f} ({n_future_stdev_second:>5.2f})")

In [70]:
def LR_runs(df, n_days):    
    df.reset_index(drop=True, inplace=True)
        
    # Initialize X and Y
    X = []
    Y = []
    
    # Populate X and Y
    # n = max(n_days, n_sma)
#     print(df[['Adj Close','SMA']].head())
#     for i in range(n_days*n_sma, n_days*n_sma+5):
    for i in range(n_days, len(df)):
        #print("i: ", i)
        X.append(df[['Log Returns']].iloc[i-n_days:i].values.flatten())
        Y.append(df['Log Returns'].iloc[i])
#     print(f"len(X)= {len(X)}, X[{i}]: {X}")
#     print(f"len(Y)= {len(Y)}, Y[{i}]: {Y}")
#     print()
#     print(df['SMA'].head(10))

    # split data
    test_size = int(len(X) * 0.2)

    # Training set
    X_train = X[:-test_size]
    y_train = Y[:-test_size]

    # Testing set
    X_test = X[-test_size:]
    y_test = Y[-test_size:]

    # Fit the model on the training data
    model = LinearRegression().fit(X_train, y_train)
    # normalize = False (input data)
    # copy_x = True (overwrite input variables)
    # n_jobs = None (number of parallelism. -1 uses all available processors)

    r_sq = model.score(X_test, y_test)
    print(f"coefficient of determination: {r_sq}")
    print(f"intercept: {model.intercept_}")
    print(f"slope: {model.coef_}")
    return r_sq

In [5]:
def LR_EMA(df, n_days, n_ema):    
    # Calculate Exponential Moving Average (EMA)
    df['EMA'] = df['Log Returns'].ewm(span=n_ema, adjust=False).mean()

    df = df.dropna(subset=['Log Returns', 'EMA'])
    df.reset_index(drop=True, inplace=True)
    
    # Initialize X and Y
    X = []
    Y = []
    
    # Populate X and Y
    for i in range(n_days*n_ema, len(df)):
        #print("i: ", i)
        X.append(df[['EMA']].iloc[i-n_days*n_ema:i:n_ema].values.flatten())
        Y.append(df['EMA'].iloc[i])

    # split data
    test_size = int(len(X) * 0.2)

    # Training set
    X_train = X[:-test_size]
    y_train = Y[:-test_size]

    # Testing set
    X_test = X[-test_size:]
    y_test = Y[-test_size:]

    # Fit the model on the training data
    model = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    # normalize = False (input data)
    # copy_x = True (overwrite input variables)
    # n_jobs = None (number of parallelism. -1 uses all available processors)

    r_sq = model.score(X_test, y_test)
#     print(f"coefficient of determination: {r_sq}")
#     print(f"intercept: {model.intercept_}")
#     print(f"slope: {model.coef_}")
    return r_sq

In [3]:
def LR_SMA(df, n_days, n_sma):    
    # Calculate the simple moving average (SMA) for a window of n days
    df['SMA'] = df['Log Returns'].rolling(window=n_sma).mean()

    df = df.dropna(subset=['Log Returns', 'SMA'])
    df.reset_index(drop=True, inplace=True)
    
    # Initialize X and Y
    X = []
    Y = []
    
    # Populate X and Y
    for i in range(n_days*n_sma, len(df)):
        #print("i: ", i)
        X.append(df[['SMA']].iloc[i-n_days*n_sma:i:n_sma].values.flatten())
#         X.append(df[['SMA']].iloc[i-n_days*n_sma:i:n_sma].values.flatten())
        Y.append(df['SMA'].iloc[i])

    # split data
    test_size = int(len(X) * 0.2)

    # Training set
    X_train = X[:-test_size]
    y_train = Y[:-test_size]

    # Testing set
    X_test = X[-test_size:]
    y_test = Y[-test_size:]

    # Fit the model on the training data
    model = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    # normalize = False (input data)
    # copy_x = True (overwrite input variables)
    # n_jobs = None (number of parallelism. -1 uses all available processors)

    r_sq = model.score(X_test, y_test)
#     print(f"coefficient of determination: {r_sq}")
#     print(f"intercept: {model.intercept_}")
#     print(f"slope: {model.coef_}")
    return r_sq

In [25]:
def LR_tomorrow(df, n_days, n_ma, n_future):  
    
    df_tech = technical_indicators(df, n_ma, n_future)
    
    # Initialize X and Y
    X = []
    Y = []
    
    # Populate X and Y
    for i in range(n_days*n_ma, len(df_tech)-(-n_ma+1+n_future)):
#     for i in range(n_days*n_ma, n_days*n_ma + 45):
        emas = df_tech[['EMA']].iloc[i-n_days*n_ma:i:n_ma].values.flatten()
        X.append(emas)
        y_SMA = df['y_SMA'].iloc[i - n_ma + n_future]
        Y.append(y_SMA)

    # split data
    test_size = int(len(X) * 0.2)

    # Training set
    X_train = X[:-test_size]
    y_train = Y[:-test_size]

    # Testing set
    X_test = X[-test_size:]
    y_test = Y[-test_size:]
    
    # Fit the model on the training data
    model = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    # normalize = False (input data)
    # copy_x = True (overwrite input variables)
    # n_jobs = None (number of parallelism. -1 uses all available processors)

    r_sq = model.score(X_test, y_test)
#     print(f"coefficient of determination: {r_sq}")
#     print(f"intercept: {model.intercept_}")
#     print(f"slope: {model.coef_}")
    return r_sq

In [4]:
def prepare_data_old(df, n_days, n_ma, n_future):
    df = technical_indicators(df, n_ma, n_future)
#     print("df.head: ", df.head())
#     print("df.head: ", df.head())
    
    # Initialize X and Y
    X = []
    Y = []
    
    # Populate X and Y
    for i in range((n_days-1)*n_ma, len(df)-n_future):
#     for i in range(n_days*n_ma, n_days*n_ma + 55):
        emas = df[['EMA']].iloc[i-(n_days-1)*n_ma:i+1:n_ma].values.flatten()
        macd_value = df['MACD'].iloc[i]
        signal_line = df['Signal_Line'].iloc[i]
        macd_rate_of_change = df['MACD_Rate_of_Change'].iloc[i]
        macd_crossover_up = df['MACD_Crossover_Up'].iloc[i]
        macd_crossover_down = df['MACD_Crossover_Down'].iloc[i]

        features = list(emas) + [macd_value, signal_line, macd_rate_of_change, macd_crossover_up, macd_crossover_down]
        X.append(features)

        y_SMA = df['y_SMA'].iloc[i+n_future]
        Y.append(y_SMA)
    return X, Y